In [103]:
import pandas as pd
import dateutil.parser
import re
import numpy as np

In [104]:
df = pd.read_csv('../Created CSVs/all_seins_all_data_standard_dates.csv')

In [105]:
df['Date'] =  [dateutil.parser.parse(date) for date in list(df['Date'])]

In [106]:
def fix_ph(pH_li):
    pH_list  = []
    pH_li = pH_li.fillna('')
    pH_lists  = [re.findall('\d+\.*', x ) for x in pH_li]

    for pH in pH_lists:
        if len(pH) == 0:
            pH_list.append(np.NaN)
        elif len(pH) ==  1:
            pH_list.append(pH[0])
            #print pH[0]
        elif pH[0][-1] == '.':
            pH_list.append(pH[0]+pH[1])
            #print pH[0]+pH[1]
        else:
            pH_list.append(pH[0])
            #print 'bad',pH[1], pH
    #pHCompare['pH'] = pH_list
    return [float(x) for x in pH_list]

In [107]:
df['Corrected pH'] = fix_ph(df['pH'])

In [108]:
def fix_water_temp(h20_temp_list):
    water_temp_list = []
    h20_temp_list = h20_temp_list.fillna('')
    h20_temp_list  = [re.findall('\d+\.*', x ) for x in h20_temp_list]
    for temp in h20_temp_list:
        #print temp
        if len(temp) == 0:
            water_temp_list.append(np.NaN)
        elif len(temp) ==  1:
            if int(temp[0]) < 30:
                new_temp = int(temp[0]) * 9.0/5 + 32
                water_temp_list.append(str(new_temp))
            else:
                water_temp_list.append(temp[0])
            #print pH[0]
        elif temp[0][-1] == '.':
            water_temp_list.append(temp[0]+temp[1])
            #print pH[0]+pH[1]
        else:
            water_temp_list.append(temp[0])
    
    return [float(x) for x in water_temp_list]

In [109]:
df["Corrected Water Temp:"] = fix_water_temp(df['Water Temp:'])

In [110]:
def fix_salinity(sal_list):
    salinity = []
    for sal in sal_list:
        if sal != sal:
            salinity.append(sal)
        elif any(char.isdigit() for char in sal) == False:
            salinity.append(np.NAN)
        else:
            first_num  = re.search(r'\d+', sal).group()
            salinity.append(first_num)
    return [float(x) for x in salinity]

In [111]:
df['Corrected Salinity'] = fix_salinity(df['Salinity'])

In [112]:
def fix_DO(DO_list):
    cleaned_list = []
    DO_list = DO_list.fillna('')
    DO_list  = [re.findall('\d+\.*', x ) for x in DO_list]

    for DO in DO_list:
        if len(DO) == 0:
            cleaned_list.append(np.NaN)
        else:
            cleaned_list.append(float(DO[0]))
       
    return cleaned_list

In [113]:
df['Corrected DO'] = fix_DO(df['DO'])

In [114]:
df['Corrected Quantity'] = df['Quantity'].fillna(1)

In [115]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [116]:
def fix_quantity(quant_list):
    num_caught_list = []
    for num in list(quant_list):
        if type(num) ==  int:
            num_caught_list.append(num)
        elif is_number(num):
            num_caught_list.append(int(float(num)))
        else:
            number = ''
            started = False
            for char in num:
                if char.isdigit():
                    number += char
                    started = True
                else:
                    if char == '~':
                        print (started)
                    if started:
                        break
            if len(number):
                num_caught_list.append(int(number))
            else:
                if num in ['Numerous', 'TMTC', 'Too Many To Count']:
                    num_caught_list.append(30)
                else:
                    num_caught_list.append(1)
    return num_caught_list

In [117]:
df['Corrected Quantity'] = fix_quantity(df['Corrected Quantity'])

False
False
False


In [118]:
def fix_hauls(hauls_li):
    #?, Many, and Unknown have all been gotten rid of and not replaced by a number.
    clean_hauls = []
    for haul in hauls_li:
        if haul != haul:
            clean_hauls.append(np.nan)
        elif any(char.isdigit() for char in haul):
            first_num  = re.search(r'\d+', haul).group()
            clean_hauls.append(first_num)
        else:
            clean_hauls.append(np.nan)
    return clean_hauls

In [119]:
df['Corrected # Hauls'] = fix_hauls(df['# Hauls'])

In [120]:
remove_row_list = ['two hemigrapsis molts were found on the beach', 'species caught', 'slipper snail shell', 'slipper snail shell', 'slipper shell', 'oyster shell','hemigraphis molt']

In [121]:
fish_list = [re.sub(r'([^\s\w]|_)+', '', f) for f in df['Common Name']]
fish_list = [f.lower() for f in fish_list]
fish_list = [f.strip() for f in fish_list]

In [122]:
df['Common Name'] = fish_list

In [123]:
df = df[df['Common Name'].isin(remove_row_list) == False]

In [124]:
df = df.reset_index()

In [125]:
fish_list = [f.replace('yoy', '') for f in df['Common Name']]

In [126]:
from difflib import SequenceMatcher

In [127]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [128]:
for i in range(len(fish_list)):
    if fish_list[i] == "pip fish":
        fish_list[i] = "pipefish"

In [129]:
count = 0
fish_changes = []
for i in range(len(fish_list)):
    for j in range(i+1, len(fish_list)):
        if fish_list[i] ==  fish_list[j]:
            continue
        if similar(fish_list[i], fish_list[j]) > 0.85:
            fish_changes.append([fish_list[i], fish_list[j]])
            if len(fish_list[i]) < len(fish_list[j]):
                fish_list[j] = fish_list[i]
            else:
                fish_list[i] = fish_list[j]

In [130]:
len(set(fish_list))

193

In [131]:
### Check for fish that have repeats in the excel document. Aka like the sea robin.

In [132]:
def overall_name_changes(fish_list):
    name_changes = {
                    'eel american n':'american eel',
                    "anchovy": "unidentified anchovy",
                        'anchovy bay n': 'bay anchovy',
                        'anchovy striped n': 'striped anchovy',
                    'shore crab': 'asian shore crab',
                    "menhaden": "unidentified menhaden",
                        'menhaden atlantic n':'atlantic menhaden',
                    'needlefish atlantic n':'atlantic needlefish',
                    'atlantic silverside menidia menidia': 'atlantic silverside','atlanticsilverside': 'atlantic silverside','silverside atlantic n':'atlantic silverside',
                    "silverside": "atlantic silverside", 
                    'tomcod': 'atlantic tomcod','tomcod atlantic n': 'atlantic tomcod',
                    'bass': 'unidentified bass',
                        'juvenille stripped bass': 'striped bass', 'jvvenilee striped buss': 'striped bass', 'bass striped n':'striped bass',
                        "sea bass": "black sea bass",
                    'bluecrab': 'blue crab', 'crab blue crab':'blue crab', ' crab blue crab': 'blue crab',
                    'bluefish juvenile': 'bluefish',
                    "mussel": "unidentified mussel",
                        'blue muscle': 'blue mussel',
                    "clams":"unidentified clam", "clam": "unidentified clam",
                        'softshell clam steamer clam':'softshell clam','soft shell clam':'softshell clam',
                    'comb jelly': 'unidentified comb jelly','comb jellies': 'unidentified comb jelly','comb jellies ctenophores': 'unidentified comb jelly', 'ctenophore':'unidentified comb jelly','20 ctenophores': 'unidentified comb jelly',
                    'cunner bergall chogy n':'cunner', 
                    "oyster":"eastern oyster",
                    'larval fish': 'unidentified fish',
                    "flatfish": "unidentified flatfish",
                        "flounder":"unidentified flounder",
                            "flounder righteye": "winter flounder",'winter flounder righteye': 'winter flounder','flounder winter n': 'winter flounder',
                            'window pain':'windowpane flounder',
                    'ghost shrimp':'glass shrimp','glass shrimp shore shrimp': 'glass shrimp',
                    'golden star colonial tunicate': 'golden star tunicate','star tunicate':'golden star tunicate',
                    'green crab carcinus maenas': 'green crab','green crab european':'green crab',
                    'sculpin': 'unidentified sculpin',
                        'grubby sculpin family': 'grubby',
                    "hermit crab": "unidentified hermit crab",
                        'hermit crab longwristed':'long clawed hermit crab',
                    'herring spp pre metamorph': 'unidentified herring',"herring": "unidentified herring",
                        "blue back": "blueback herring",'herring blueback n':'blueback herring',
                        'herring atlantic n':'atlantic herring',
                        ' american shad': 'american shad',
                    'horseshoe crab atlantic': 'horseshoe crab',
                    "hydra medusa": "hydrozoa", "hydroid": "hydrozoa", "hydromedusae": "hydrozoa",
                    "jelly fish": "unidentified jelly fish",'true jellies': 'unidentified jelly fish','true jelly': 'unidentified jelly fish', 'true jelly fish': 'unidentified jelly fish',
                        'lions mane jellyfish cyanea capillata': 'lions mane jellyfish',
                        'hydromedusamoon jelly':'moon jelly',
                    'mud crab': 'unidentified mud crab',
                    'muslces':'unidentified mussel',
                        'mussel blue':'blue mussel',
                        'mussel ribbed':'ribbed mussel',
                    'norther pipefish': 'northern pipefish', 'pipefish syngnathinae': 'northern pipefish',"pipefish":"northern pipefish",'pipefish northern n': 'northern pipefish',
                    'the cutest northern puffer ever':'northern puffer','norther puffer': 'northern puffer', 'puffer': 'northern puffer','puffer northern n':'northern puffer',
                    "sea horse hippocampus": "northern seahorse", "sea horse": "northern seahorse","seahorse": "northern seahorse",'lined seahorse':'northern seahorse',
                    "sea robin": "northern sea robin",
                    'spot lafayette n':'spot',
                    '1 oyster toadfish caught in traffic cone' : 'oyster toadfish','toadfish oyster n': 'oyster toadfish',  
                    'porgy':'unidentified porgy', 'porgies':'unidentified porgy',
                        'porgyscup': 'scup', 
                    "poly worm": "unidentified polychaete worm",
                       'clam worm sandworm': 'sandworm', 'sand worm': 'sandworm','clam worm probably nereis spp':'sandworm',
                    "sea snail": "unidentified sea snail","snail": "unidentified sea snail","snail eggs": "unidentified sea snail",
                        'periwinkle common': 'common periwinkle snail', 'periwinkle': 'common periwinkle snail', 'periwinkle snail': 'common periwinkle snail',
                        'mudsnail eastern': "eastern mud snail",
                        'oyster drill': 'oyster drill snail',
                    "shrimp": "unidentified shrimp","shrimp shore and mud": "unidentified shrimp",
                        'shrimp seven spine': 'sand shrimp',"grass shrimp": "sand shrimp",'seven spine shrimp': 'sand shrimp',
                        'common shore shrimp': 'shore shrimp', "glass shrimp": "shore shrimp",'grass shrimp shore shrimp':'shore shrimp',
                    'perch silver n': 'silver perch',
                    'skillet fish': 'skilletfish',
                    "sponge": "unidentified sponge",
                    'common spider crab': 'spider crab',"decoration crab": "spider crab",
                    'blackfish tautog': 'tautog', 'taotog': 'tautog','tautogblackfish': 'tautog',
                    "tunicates": "tunicate", "tunicate sponge": "tunicate",
                        'sea squirtsolitary tunicate': 'sea squirt',
                    'perch white n': 'white perch',}
    for i in range(len(fish_list)):
        if fish_list[i] in name_changes:
            fish_list[i] = name_changes[fish_list[i]]
    return fish_list
    

In [133]:
# Should have replaced all the make_name_corrected_names

In [134]:
def make_name_corrected_names(fish_list):
    name_changes = {'1 oyster toadfish caught in traffic cone' : 'oyster toadfish', '20 ctenophores': 'ctenophore',
               'atlantic silverside menidia menidia': 'atlantic silverside','atlanticsilverside': 'atlantic silverside',
               'blackfish tautog': 'tautog', 'blue muscle': 'blue mussel', 'bluefish juvenile': 'bluefish',
               'comb jellies': 'comb jelly','comb jellies ctenophores': 'comb jelly','common shore shrimp': 'shore shrimp',
               'common spider crab': 'spider crab', 'bluecrab': 'blue crab', 'crab blue crab':'blue crab', ' crab blue crab': 'blue crab',
               'juvenille stripped bass': 'striped bass', 'jvvenilee striped buss': 'striped bass', 'taotog': 'tautog',
               'tautogblackfish': 'tautog', 'the cutest northern puffer ever':'northern puffer','window pain':'windowpane flounder',
               'true jellies': 'jelly fish', 'true jelly': 'jelly fish', 'true jelly fish': 'jelly fish', 'sea squirtsolitary tunicate': 'sea squirt',
               'oyster drill': 'oyster drill snail', 'green crab carcinus maenas': 'green crab','green crab european':'green crab',
               'porgyscup': 'porgy', 'scup': 'porgy', 'norther puffer': 'northern puffer', 'norther pipefish': 'northern pipefish', 'ctenophore':'comb jelly',
                'ghost shrimp':'glass shrimp','glass shrimp shore shrimp': 'glass shrimp', 'golden star colonial tunicate': 'golden star tunicate',
               'lions mane jellyfish cyanea capillata': 'lions mane jellyfish', 'muslces':'mussel', 'pipefish syngnathinae': 'northern pipefish',
               "porgies":"porgy", 'lined seahorse':'northern seahorse',"sea horse hippocampus": "northern seahorse", "sea horse": "northern seahorse", 'winter flounder righteye': 'winter flounder',
               'shrimp seven spine': 'seven spine shrimp', 'star tunicate':'golden star tunicate', 'seven spine shrimp': 'sand shrimp',
               'puffer': 'northern puffer', 'northern sea robin': 'sea robin', 'herring spp pre metamorph': 'herring', 'grubby sculpin family': 'grubby',
               "blue back": "blueback herring", "anchovy": "bay anchovy", "decoration crab": "spider crab", "flatfish": "unidentified flatfish","flounder":"unidentified flatfish",
               "flounder righteye": "winter flounder", "glass shrimp": "shore shrimp", "grass shrimp": "sand shrimp","grubby sculpin family": "grubby",
               "hermit crab": "unidentified hermit crab", "herring": "unidentified herring", "herring spp pre metamorph":"unidentified herring",
               "hydra medusa": "hydrozoa", "hydroid": "hydrozoa", "hydromedusae": "hydrozoa", "jelly fish": "unidentified jelly fish", "menhaden": "atlantic menhaden",
               "mussel": "unidentified mussel", "oyster":"eastern oyster", "pipefish":"northern pipefish", "poly worm": "sand worm", "sea bass": "black sea bass",
               "sea robin": "northern sea robin", "sea snail": "unidentified sea snail", "seahorse": "northern seahorse", "shrimp": "unidentified shrimp",
               "shrimp shore and mud": "unidentified shrimp", "silverside": "atlantic silverside", "snail": "unidentified snail","snail eggs": "unidentified snail eggs",
               "sponge": "unidentified sponge", "tunicates": "tunicate", "tunicate sponge": "tunicate",' american shad': 'american shad', 'shore crab': 'asian shore crab',
                   'tomcod': 'atlantic tomcod', 'skillet fish': 'skilletfish', 'bass': 'striped bass', "clams":"clam",'flounder winter n': 'winter flounder',
                   'herring atlantic n':'atlantic herring','herring blueback n':'blueback herring','mudsnail eastern': "mud snail",'mussel blue':'blue mussel','mussel ribbed':'ribbed mussel',
                   'periwinkle common': 'periwinkle','puffer northern n':'northern puffer','hermit crab longwristed':'long clawed hermit crab'}
    for i in range(len(fish_list)):
        if fish_list[i] in name_changes:
            fish_list[i] = name_changes[fish_list[i]]
    return fish_list

In [135]:
def make_name_corrected_and_grouped_names(fish_list):
    name_changes = {'1 oyster toadfish caught in traffic cone' : 'oyster toadfish', '20 ctenophores': 'comb jelly',
               'atlantic silverside menidia menidia': 'atlantic silverside','atlanticsilverside': 'atlantic silverside',
               'blackfish tautog': 'tautog', 'blue muscle': 'mussel', 'bluefish juvenile': 'bluefish',
               'comb jellies': 'comb jelly','comb jellies ctenophores': 'comb jelly','common shore shrimp': 'shrimp',
               'common spider crab': 'spider crab', 'bluecrab': 'blue crab', 'crab blue crab':'blue crab', ' crab blue crab': 'blue crab', 
               'juvenille stripped bass': 'striped bass', 'jvvenilee striped buss': 'striped bass', 'taotog': 'tautog',
               'tautogblackfish': 'tautog', 'the cutest northern puffer ever':'puffer','window pain':'flatfish',
               'true jellies': 'jellyfish', 'true jelly': 'jellyfish', 'true jelly fish': 'jellyfish', 'sea squirtsolitary tunicate': 'tunicates',
               'oyster drill': 'sea snail', 'green crab carcinus maenas': 'green crab','green crab european':'green crab',
               'porgyscup': 'porgy', 'scup':'porgy', 'norther puffer': 'puffer', 'norther pipefish': 'northern pipefish', 'ctenophore':'comb jelly',
                'ghost shrimp':'shrimp','glass shrimp shore shrimp': 'shrimp', 'golden star colonial tunicate': 'tunicates',
               'lions mane jellyfish cyanea capillata': 'jellyfish', 'muslces':'mussel', 'pipefish syngnathinae': 'northern pipefish',
               "porgies":"porgy",'lined seahorse':'northern seahorse',"seahorse":"northern seahorse", "sea horse hippocampus": "northern seahorse", "sea horse": "northern seahorse", 'winter flounder righteye': 'flatfish',
               'shrimp seven spine': 'shrimp', 'star tunicate':'tunicates', 'seven spine shrimp': 'shrimp','pipefish': 'northern pipefish',
               'northern puffer': 'puffer', 'northern sea robin': 'sea robin', 'herring spp pre metamorph': 'herring', 'grubby sculpin family': 'grubby',
               "blue back": "herring", 'alewife':'herring', 'atlantic herring': 'herring', 'blueback herring': 'herring',
               'colonial tunicate': 'tunicates', 'golden star tunicate': 'tunicates','sea squirt': 'tunicates', 'tunicate sponge': 'tunicates',
               'shrimp shore and mud': 'shrimp', 'seven spine shrimp': 'shrimp', 'shore shrimp': 'shrimp', 'sand shrimp': 'shrimp', 'mantis shrimp': 'shrimp',
               'grass shrimp': 'shrimp', 'glass shrimp': 'shrimp', 'flounder': 'flatfish', 'flounder righteye': 'flatfish', 'hogchoker': 'flatfish',
               'summer flounder': 'flatfish', 'windowpane flounder': 'flatfish', 'winter flounder': 'flatfish', 'menhaden': 'atlantic menhaden',
               'silverside': 'atlantic silverside', 'blue mussel': 'mussel', 'eastern oyster': 'oyster', 'anchovy': 'bay anchovy',
                'moon jelly': 'jellyfish', 'jelly fish': 'jellyfish', 'lions mane jellyfish': 'jellyfish', 'scud': 'amphipod', 'ribbed mussel': 'mussel',
               'mud snail': 'sea snail', 'periwinkle': 'sea snail', 'oyster drill snail': 'sea snail', 'snail': 'sea snail', 'whelk snail':'sea snail', 'snail eggs': 'sea snail', 'sea bass': 'black sea bass',
               'sheepshead porgy': 'porgy', 'striped searobin': 'sea robin', 'black fingered mud crab': 'mud crab', 'clam worm probably nereis spp': 'polychaete worm',
               'poly worm': 'polychaete worm', 'sandworm': 'polychaete worm', 'white fingered mud crab': 'mud crab', 'decoration crab': 'spider crab',
               'hydra medusa': 'hydrozoa', 'hydroid': 'hydrozoa', 'hydromedusae' : 'hydrozoa', 'red beard sponge': 'sponge', ' american shad': 'american shad',
                   "long clawed hermit crab": "hermit crab", 'shore crab': 'asian shore crab', 'tomcod': 'atlantic tomcod', 'skillet fish': 'skilletfish', 'pin fish': 'porgy', 'bass': 'striped bass',
                   "clams": "clam",'flounder winter n': 'flatfish','grass shrimp shore shrimp':'shrimp','herring atlantic n':'herring','herring blueback n':"herring",'mudsnail eastern': "sea snail",
                   'mussel blue':'mussel','mussel ribbed':'mussel','periwinkle common':'sea snail','puffer northern n':'puffer','hermit crab longwristed':'hermit crab','undetermined jelly':'jellyfish',
                   "comb jelly beroe": "comb jelly", "comb jelly leidyi": "comb jelly", "hydromedusamoon jelly": "jellyfish", "hydromedusa":"jellyfish", "larval fish": "unidentified fish" }
    for i in range(len(fish_list)):
        if fish_list[i] in name_changes:
            fish_list[i] = name_changes[fish_list[i]]
    return fish_list

In [136]:
def correcting_scientific(f_list):
    name_changes = {'anchovy bay n': 'bay anchovy', 'anchovy striped n': 'striped anchovy','bass striped n':'striped bass',
          'cunner bergall chogy n':'cunner', 'clam worm sandworm': 'polychaete worm','eel american n':'american eel',
                    'horseshoe crab atlantic': 'horseshoe crab', 'menhaden atlantic n':'atlantic menhaden','needlefish atlantic n':'needlefish',
                    'perch silver n': 'silver perch', 'perch white n': 'white perch','pipefish northern n': 'northern pipefish','silverside atlantic n':'atlantic silverside',
                    'softshell clam steamer clam':'clam','spot lafayette n':'spot','toadfish oyster n': 'oyster toadfish', 'tomcod atlantic n': 'atlantic tomcod','soft shell clam':'clam',
                    'softshell clam':'clam'
                   }
    for i in range(len(f_list)):
        if f_list[i] in name_changes:
            f_list[i] = name_changes[f_list[i]]
    return f_list

In [137]:
nogroup = make_name_corrected_names(fish_list[:])

In [138]:
group = make_name_corrected_and_grouped_names(fish_list[:])

In [139]:
nogroup = correcting_scientific(nogroup)

In [140]:
group = correcting_scientific(group)

In [141]:
df['Grouped Corrected Common Name'] = group#df2['Species']

In [142]:
df['Corrected Common Name'] = nogroup#df3['Species']

#### not yet cleaned: Time, Weather & Air Temp, Tide Times, Tide is Moving, Turbidity, # student seiners, # visitors, instructors, Scientific Name, Brought to Ed Center, Other Info, Notes from Seining
#### Of these Weather & Air Temp, Tide Times Tide is Moving and Turbidity should be done. The rest probably don't matter
#### I'd also like to see if we can pull from outside data for Weather & Air Temp, Water Temp, Tide Times, Tide is Moving, DO, pH, Salinity, and Turbidity. If so we could get better readings.

#### Time should be start time, end time, length of seine - 3 columns, Weather and Air Temp should be 2 Weather and Air Temp, plus both from other origin.

#### if you wanted to make a  further interesting and easy to work with csv, you can make one that combines all the animals caught list, and amount caught into one row with a comma separated string.

In [143]:
df = df.drop(['index', 'Unnamed: 0'], axis=1)

In [144]:
def find_dif(list1,list2):
    for item in list1:
        if item not in list2:
            print (item)
            
    for item in list2:
        if item not in list1:
            print (item)

In [145]:
df = df[['Date','Time','Weather & Air Temp:','Water Temp:','Corrected Water Temp:','Water Temp Probe','pH','Corrected pH','pH Probe','DO',
              'Corrected DO','DO Probe','Turbidity','Turbidity Probe (NTU)','Salinity',
              'Corrected Salinity','Salinity Probe','Tide is moving','Tide Times','# Hauls','Corrected # Hauls','Common Name', 'Corrected Common Name', 
              'Grouped Corrected Common Name', 'Quantity', 
             'Corrected Quantity', 'Scientific Name',
            'Instructors/ Seine Staff:','# student seiners (CLASSES ONLY)',
             '# of visitors (Public Seine ONLY)', '# Brought to Ed Center',
        'Other info (Gender, Length, Dead/Alive, other physical description)',
            'NOTES FROM SEINEING SEASON:','NOTES FROM SEINE:']].sort_values(by = 'Date').reset_index().drop('index', axis=1)

In [146]:
df.to_csv('../Created CSVs/Cleaned_For_Analysis_All_Data.csv')